In [1]:
import numpy as np
import var
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
import pandas as pd
from scipy.linalg import inv, kron 

In [2]:
np.random.seed(0)
data = np.cumsum(np.random.randn(100, 2), axis=0)
rdata2 = [
    [2.1, 5.2],
    [1.8, 5.3],
    [2.0, 5.1],
    [1.9, 5.4],
    [2.2, 5.2]
]
data2 = np.array(rdata2)
data2.shape

(5, 2)

In [27]:
ols = var.VAR(1)
ols_fit = ols.fit(data, 'OLS')

In [28]:
ols.coef_

array([[[ 0.95169316, -0.00642169],
        [-0.01268395,  0.98676743]]])

In [5]:
Y, Z = var.create_lag_matrix(data2, 1)

In [6]:
var.ols_estimation(Y, Z)

array([[-1.8875,  6.9   ],
       [-0.375 ,  0.5   ],
       [ 0.875 , -0.5   ]])

In [7]:
Z

array([[1. , 2.1, 5.2],
       [1. , 1.8, 5.3],
       [1. , 2. , 5.1],
       [1. , 1.9, 5.4]])

In [8]:
Y

array([[1.8, 5.3],
       [2. , 5.1],
       [1.9, 5.4],
       [2.2, 5.2]])

In [9]:
cov_matrix = ols.cov_matrix_
cov_matrix

array([[ 1.01758703, -0.03690788],
       [-0.03690788,  1.055689  ]])

In [10]:
var.gls_estimation(Y, Z, cov_matrix)

array([[-1.78459691,  6.96860206],
       [-0.3892921 ,  0.49047194],
       [ 0.8607079 , -0.50952806]])

In [11]:
n_samples, n_features = Y.shape
Y.shape

(4, 2)

In [12]:
kron_cov = kron(np.eye(len(Y)), cov_matrix)
kron_cov

array([[ 1.01758703, -0.03690788,  0.        , -0.        ,  0.        ,
        -0.        ,  0.        , -0.        ],
       [-0.03690788,  1.055689  , -0.        ,  0.        , -0.        ,
         0.        , -0.        ,  0.        ],
       [ 0.        , -0.        ,  1.01758703, -0.03690788,  0.        ,
        -0.        ,  0.        , -0.        ],
       [-0.        ,  0.        , -0.03690788,  1.055689  , -0.        ,
         0.        , -0.        ,  0.        ],
       [ 0.        , -0.        ,  0.        , -0.        ,  1.01758703,
        -0.03690788,  0.        , -0.        ],
       [-0.        ,  0.        , -0.        ,  0.        , -0.03690788,
         1.055689  , -0.        ,  0.        ],
       [ 0.        , -0.        ,  0.        , -0.        ,  0.        ,
        -0.        ,  1.01758703, -0.03690788],
       [-0.        ,  0.        , -0.        ,  0.        , -0.        ,
         0.        , -0.03690788,  1.055689  ]])

In [13]:
len(Z)

4

In [14]:
Z_tilde = kron(np.eye(n_features), Z)
Y_tilde = Y.flatten(order='F')

In [15]:
Y_tilde

array([1.8, 2. , 1.9, 2.2, 5.3, 5.1, 5.4, 5.2])

In [16]:
Z_tilde

array([[1. , 2.1, 5.2, 0. , 0. , 0. ],
       [1. , 1.8, 5.3, 0. , 0. , 0. ],
       [1. , 2. , 5.1, 0. , 0. , 0. ],
       [1. , 1.9, 5.4, 0. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 2.1, 5.2],
       [0. , 0. , 0. , 1. , 1.8, 5.3],
       [0. , 0. , 0. , 1. , 2. , 5.1],
       [0. , 0. , 0. , 1. , 1.9, 5.4]])

In [17]:
Z_T_cov_inv = np.dot(Z_tilde.T, inv(kron_cov)) 

In [18]:
ZT_COV_Z = Z_T_cov_inv @ Z_tilde

In [19]:
ZT_COV_Y = Z_T_cov_inv @ Y_tilde

In [20]:
Beta_GLS = inv(ZT_COV_Z) @ ZT_COV_Y

In [21]:
Beta_GLS

array([-1.78459691, -0.3892921 ,  0.8607079 ,  6.96860206,  0.49047194,
       -0.50952806])

In [22]:
data2 = pd.DataFrame(data2, columns=['variable_1', 'variable_2'])

# Fit the VAR model using statsmodels
model = VAR(data2)
results = model.fit(maxlags=1)
results.intercept

array([-1.8875,  6.9   ])

In [23]:
results.coefs

array([[[-0.375,  0.875],
        [ 0.5  , -0.5  ]]])

In [24]:
data2

,variable_1,variable_2
0,2.1,5.2
1,1.8,5.3
2,2.0,5.1
3,1.9,5.4
4,2.2,5.2
